# SD201 project

# Recommendations based on hours played : k-nn on players
The aim of the algorithm is to predict a list of games someone may like knowing how much he played to other games.
It means that the feature we want to predict is a list of games, and the features used to do so are, for each game in the database, the amount of hours spent playing this game.

We then want X, Y like this :  
X = [0,0,...,0,5.2,0,10,...]  
Y = [0,1,1,0,0,1,...] (the one-encoded vector of the games liked) 

To determine the k neighbors, we will use the L2 distance on the vectors of hours played for each game.  
With this distance, firstly people that played a lot the same games will be near from each other, but it will also cluster types of players : those who play a lot of different games will be together, and the niche player will be separated.  

To make a prediction, we will compute using this distance the k neighbors of a player, and make a prediction.  
For this prediction, the k-nn algorithm usually choses the majority class of the neighbors. Thus, we could define a class as the one-hot-encoding of the games played by the user. However, this may lead to 1 class per neighbor, as it is highly possible that every player in the neighborhood, even if they are close one to each other, have tried a game the other ones never played to.  
That's why we choose as a result to give the union of all the games played by the neighbors. This will guarantee that the games resulting will be chosen because they are characteristical of the neighborhood.  
Obviously, we will remove from the result the games already played by the initial player (we don't want to recommand to someone a game he already played for 200 hours...)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Get the data

#read data
data = pd.read_csv('steam-200k.csv')
#clean data
data.columns = ['id','game','state','hours_played','0']
data = data.drop('0',axis=1)
played_games = data.loc[data['state']=='play']

#Standardization
standardization_dict = dict()
for game_name, s in played_games.groupby('game')['hours_played']:
    standardization_dict[game_name]=dict()
    series = s[s>0.0]  #take only games played
    standardization_dict[game_name]['average'] = series.mean()
    if series.std() > 0 : #for some games that are not much played, std is 0 which creates errors
        standardization_dict[game_name]['std'] = series.std()
    else:
        standardization_dict[game_name]['std'] = 1e-8

def standardize(game, hours):
    return (hours - standardization_dict[game]['average'])/standardization_dict[game]['std']
played_games['hours_played'] = played_games.apply(lambda x : standardize(x.game, x.hours_played),axis=1)

/tmp/ipykernel_404441/1935983977.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  played_games['hours_played'] = played_games.apply(lambda x : standardize(x.game, x.hours_played),axis=1)


In [3]:
played_games.head()

,id,game,state,hours_played
0,151603712,The Elder Scrolls V Skyrim,play,0.970886
2,151603712,Fallout 4,play,0.294820
4,151603712,Spore,play,-0.193861
6,151603712,Fallout New Vegas,play,-0.540694
8,151603712,Left 4 Dead 2,play,-0.223730


In [4]:
#Get a dict of games and hours played for each id
played_dict = played_games.groupby('id').apply(lambda g : dict(zip(g['game'], g['hours_played'])))
played_dict.head()

id
5250      {'Cities Skylines': 3.158903786773205, 'Deus E...
76767     {'Counter-Strike': 0.20796408562479032, 'Call ...
86540     {'The Elder Scrolls V Skyrim': 0.0478209339091...
144736              {'Counter-Strike': -0.3820160793060811}
181212    {'Counter-Strike': -0.3792674734404125, 'Half-...
dtype: object

In [5]:
#Create vectors of hours played 
hours_encoded = played_dict.apply(pd.Series)
#Replace NaN values by 0 : a game not in the dict has never been played
hours_encoded = hours_encoded.fillna(0)
#We drop the ids because they are not useful anymore
hours_encoded = hours_encoded.reset_index(drop=True)
#Sort by name of games
hours_encoded = hours_encoded.reindex(sorted(hours_encoded.columns),axis=1)
hours_encoded.head()

,007 Legends,0RBITALIS,1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby),10 Second Ninja,"10,000,000",100% Orange Juice,1000 Amps,12 Labours of Hercules,12 Labours of Hercules II The Cretan Bull,12 Labours of Hercules III Girl Power,...,rFactor,rFactor 2,realMyst,realMyst Masterpiece Edition,resident evil 4 / biohazard 4,rymdkapsel,sZone-Online,the static speaks my name,theHunter,theHunter Primal
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#Get the list of games
games_list = list(hours_encoded.columns)
games_list.sort()
len(games_list)

3600

In [7]:
#We now create our "classes" : the list of games played by each player
played_list = played_games.groupby('id')['game'].apply(list).reset_index(drop=True)
played_list.head()

0    [Cities Skylines, Deus Ex Human Revolution, Po...
1    [Counter-Strike, Call of Duty World at War, To...
2    [The Elder Scrolls V Skyrim, Audiosurf, XCOM E...
3                                     [Counter-Strike]
4             [Counter-Strike, Half-Life 2 Lost Coast]
Name: game, dtype: object

In [8]:
#And we one-hot encode them
from sklearn.preprocessing import MultiLabelBinarizer
#One-hot-encode the lists
mlb = MultiLabelBinarizer()

played_encoded = pd.DataFrame(mlb.fit_transform(played_list),
                   columns=mlb.classes_)

played_encoded.head()

,007 Legends,0RBITALIS,1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby),10 Second Ninja,"10,000,000",100% Orange Juice,1000 Amps,12 Labours of Hercules,12 Labours of Hercules II The Cretan Bull,12 Labours of Hercules III Girl Power,...,rFactor,rFactor 2,realMyst,realMyst Masterpiece Edition,resident evil 4 / biohazard 4,rymdkapsel,sZone-Online,the static speaks my name,theHunter,theHunter Primal
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#Create training and testing sets
#We just let the bought games in y values, but labels are not useful there because we will only compute the
#neighbors and find the result ourselves
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(hours_encoded, played_encoded, test_size=0.2, random_state=42)

Let's check if our data and labels are coherent

In [10]:
X_train.iloc[150]

007 Legends                                                   0.0
0RBITALIS                                                     0.0
1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby)    0.0
10 Second Ninja                                               0.0
10,000,000                                                    0.0
                                                             ... 
rymdkapsel                                                    0.0
sZone-Online                                                  0.0
the static speaks my name                                     0.0
theHunter                                                     0.0
theHunter Primal                                              0.0
Name: 10527, Length: 3600, dtype: float64

In [11]:
#To retrieve the list of played games, we get the list of games with a
#non zero time played and check the name corresponding to the index
def games_of_player(i):
    return [games_list[index] for index in list(np.asarray(y_train.iloc[i]).nonzero())[0]]

games_of_player(150)

["Assassin's Creed Revelations",
 'Borderlands',
 'Counter-Strike Global Offensive',
 'Darksiders',
 'Darksiders II',
 "Deus Ex Human Revolution - Director's Cut",
 'Fallout 3',
 'Fallout New Vegas',
 'Half-Life 2',
 'Hard Reset',
 'Hitman Codename 47',
 'Metro 2033',
 'ORION Prelude',
 'PROTOTYPE 2',
 'Remember Me']

In [12]:
X_train.iloc[150][['Counter-Strike Global Offensive','Fallout New Vegas','Borderlands']]

Counter-Strike Global Offensive    0.065582
Fallout New Vegas                 -0.378141
Borderlands                       -0.419914
Name: 10527, dtype: float64

It seems clean! Let's implement the knn algorithm

In [13]:
from sklearn.neighbors import NearestNeighbors

We will choose 100 neighbors. As we have almost 10000 users, it means we link a player to 1% of the dataset, which seems to be a fair choice.

In [14]:
neigh = NearestNeighbors(n_neighbors=100, metric='euclidean')
neigh.fit(X_train.values)

NearestNeighbors(metric='euclidean', n_neighbors=100)

In [15]:
kn = neigh.kneighbors([X_test.iloc[400]])

In [16]:
kn

(array([[0.67144898, 0.67145315, 0.67146982, 0.67206007, 0.67643122,
         0.71520583, 0.76151562, 0.77707296, 0.85966977, 0.85966977,
         0.88583856, 0.89441836, 0.9123968 , 0.9123968 , 0.91239682,
         0.91239682, 0.91239682, 0.91239682, 0.91239682, 0.91239686,
         0.91239686, 0.91239686, 0.91239686, 0.91239686, 0.91239686,
         0.91239686, 0.91239694, 0.91239694, 0.91239694, 0.91239694,
         0.91239694, 0.91239694, 0.91239694, 0.91239694, 0.91239705,
         0.91239705, 0.91239705, 0.91239705, 0.91239705, 0.91239705,
         0.91239718, 0.91239718, 0.91239718, 0.91239718, 0.91239718,
         0.91239718, 0.91239718, 0.91239718, 0.91239718, 0.91239718,
         0.91239718, 0.91239735, 0.91239735, 0.91239735, 0.91239735,
         0.91239735, 0.91239735, 0.91239735, 0.91239755, 0.91239755,
         0.91239755, 0.91239755, 0.91239755, 0.91239755, 0.91239755,
         0.91239755, 0.91239755, 0.91239755, 0.91239777, 0.91239777,
         0.91239777, 0.91239777, 0

In [17]:
games_of_player(5163)

['Dota 2']

In [18]:
X_test.iloc[400]['Dota 2']

-0.3215113027758875

In [19]:
X_train.iloc[5163]['Dota 2']

-0.3215113027758875

We notice that a lot of players have played the same popular games, and some of them have only played those ones.
In our example, the algorithm found a lot of neighbors that played the exact same time to Dota 2 than the initial player, and thus are chosen even thought they only played to Dota, which means we can't find a game to recommand.  
-> **outliers**  

Let's try to take only the players that played a minimum of 3 games.  

As we will have much less data, we will only take 5 neighbors.

In [20]:
played_dict_3 = played_dict.loc[played_dict.map(len)>=3]
played_dict_3.head()

id
5250      {'Cities Skylines': 3.158903786773205, 'Deus E...
76767     {'Counter-Strike': 0.20796408562479032, 'Call ...
86540     {'The Elder Scrolls V Skyrim': 0.0478209339091...
229911    {'Counter-Strike Condition Zero': 1.1203642472...
298950    {'Team Fortress 2': 3.045243405037884, 'Counte...
dtype: object

In [21]:
#Create vectors of hours played 
hours_encoded_3 = played_dict_3.apply(pd.Series)
#Replace NaN values by 0 : a game not in the dict has never been played
hours_encoded_3 = hours_encoded_3.fillna(0)
#We drop the ids because they are not useful anymore
hours_encoded_3 = hours_encoded_3.reset_index(drop=True)
#Sort by name of games
hours_encoded_3 = hours_encoded_3.reindex(sorted(hours_encoded_3.columns),axis=1)

#Get the list of games
games_list = list(hours_encoded_3.columns)
games_list.sort()

#We now create our "classes" : the list of games played by each player
played_list_3 = played_list.loc[played_list.map(len)>=3]

#And we one-hot encode them
mlb = MultiLabelBinarizer()

played_encoded_3 = pd.DataFrame(mlb.fit_transform(played_list_3),
                   columns=mlb.classes_)

X_train, X_test, y_train, y_test = train_test_split(hours_encoded_3, played_encoded_3, test_size=0.2, random_state=42)


#To retrieve the list of played games, we get the list of games with a
#non zero time played and check the name corresponding to the index
def games_of_player_train(i):
    return [games_list[index] for index in list(np.asarray(y_train.iloc[i]).nonzero())[0]]

def games_of_player_test(i):
    return [games_list[index] for index in list(np.asarray(y_test.iloc[i]).nonzero())[0]]

neigh = NearestNeighbors(n_neighbors=5, metric='euclidean')
neigh.fit(X_train.values)

NearestNeighbors(metric='euclidean')

In [22]:
games_of_player_test(30)

['Far Cry',
 'Far Cry 3',
 'Far Cry 3 Blood Dragon',
 'Grand Theft Auto V',
 'Team Fortress 2',
 'The Elder Scrolls V Skyrim']

In [23]:
kn = neigh.kneighbors([X_test.iloc[30]])
kn

(array([[0.79571713, 0.79902462, 0.80780405, 0.82446456, 0.8304481 ]]),
 array([[ 980, 2151, 1410, 2507, 1544]]))

In [24]:
games_of_player_train(1643)

['Loadout',
 'PlanetSide 2',
 'Team Fortress 2',
 'The Elder Scrolls V Skyrim',
 'Transformice']

In [25]:
games_of_player_train(2162)

['Fallen Earth', 'The Elder Scrolls V Skyrim', 'Warframe', 'theHunter']

The results seem pretty good : the neighbors have played games of the same genre as the initial player.  

However, if we do what we explained before : take the union of games played by the neighbors, we quickly realise that we will have a lot of games recommanded. Moreover, those recommandations may be irrelevant : if someone close played a game for only a few hours, it probably means that he didn't like it and thus we should not recommand it to the same type of player.  

That's why we will take the games **liked** by the neighbors. To do so, we will define a game as liked if a player has played more than the average time spent on the game by all the players of the dataset that have launched the game (not to take into account all the players that did not buy it).  

With our standardization, it means that we will take games with positive hours played.

In [26]:
played_dict_3 = played_dict.loc[played_dict.map(len)>=3]
played_dict_3.head()

id
5250      {'Cities Skylines': 3.158903786773205, 'Deus E...
76767     {'Counter-Strike': 0.20796408562479032, 'Call ...
86540     {'The Elder Scrolls V Skyrim': 0.0478209339091...
229911    {'Counter-Strike Condition Zero': 1.1203642472...
298950    {'Team Fortress 2': 3.045243405037884, 'Counte...
dtype: object

In [27]:
played_dict_3.iloc[2]

{'The Elder Scrolls V Skyrim': 0.047820933909160555,
 'Audiosurf': 2.5581029174126377,
 'XCOM Enemy Unknown': -0.12563492149210723,
 'Far Cry 3': -0.27559036370569656,
 'Left 4 Dead 2': -0.17294899863959529,
 'Killer is Dead': 1.2422207078794432,
 'Torchlight II': -0.3588487840957488,
 'The Walking Dead': -0.6002078303793676,
 'Max Payne 3': -0.45632355607744424,
 'Borderlands 2': -0.578650215939072,
 'Age of Empires II HD Edition': -0.4210571976531187,
 'Serious Sam HD The First Encounter': -0.9706035415493255,
 'Portal': -0.6103088782399582,
 "Tom Clancy's Ghost Recon Advanced Warfighter": -0.7071067811865477,
 'Alan Wake': -0.2525009285956324}

In [28]:
#Let's compute the liked games for each player


def dict_to_liked(dicti):
    d = dicti.copy()
    for game in d.keys():
        if d[game] > 0:
            d[game]=1
        else:
            d[game]=0
    return d

#For each player that played at least 3 games, we compute the list of game he like
liked_games = played_dict_3.map(dict_to_liked)
liked_games

id
5250         {'Cities Skylines': 1, 'Deus Ex Human Revoluti...
76767        {'Counter-Strike': 1, 'Call of Duty World at W...
86540        {'The Elder Scrolls V Skyrim': 1, 'Audiosurf':...
229911       {'Counter-Strike Condition Zero': 1, 'Call of ...
298950       {'Team Fortress 2': 1, 'Counter-Strike Global ...
                                   ...                        
306547522    {'Warface': 0, 'Dota 2': 0, 'Strife': 0, 'Team...
306971738    {'The Elder Scrolls IV Oblivion ': 0, 'Terrari...
308695132    {'La Tale': 0, 'Champions Online': 0, 'Brawlha...
308760273    {'Toribash': 0, 'Unturned': 0, 'Team Fortress ...
309404240    {'Unturned': 0, 'Team Fortress 2': 0, 'Mitos.i...
Length: 3466, dtype: object

In [29]:
#And encode them
liked_games_encoded = liked_games.apply(pd.Series)
#Replace NaN values by 0 : a game not in the dict has never been played
liked_games_encoded = liked_games_encoded.fillna(0)
liked_games_encoded = liked_games_encoded.reindex(sorted(liked_games_encoded.columns),axis=1)
liked_games_encoded

,007 Legends,0RBITALIS,1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby),10 Second Ninja,"10,000,000",100% Orange Juice,1000 Amps,12 Labours of Hercules,12 Labours of Hercules II The Cretan Bull,12 Labours of Hercules III Girl Power,...,rFactor,rFactor 2,realMyst,realMyst Masterpiece Edition,resident evil 4 / biohazard 4,rymdkapsel,sZone-Online,the static speaks my name,theHunter,theHunter Primal
id,,,,,,,,,,,,,,,,,,,,,
5250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86540,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
229911,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
298950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306547522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
306971738,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
308695132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
hours_encoded_3 = played_dict_3.apply(pd.Series)
#Replace NaN values by 0 : a game not in the dict has never been played
hours_encoded_3 = hours_encoded_3.fillna(0)
hours_encoded_3 = hours_encoded_3.reindex(sorted(hours_encoded_3.columns),axis=1)

We can then rebuild our model using liked_games as our new classes, as that is exactly what we want to predict : what games the player could like

In [31]:
#Get the list of games
games_list = list(hours_encoded_3.columns)
games_list.sort()

X_train, X_test, y_train, y_test = train_test_split(hours_encoded_3, liked_games_encoded, test_size=0.2, random_state=42)


#To retrieve the list of played games, we get the list of games with a
#non zero time played and check the name corresponding to the index
def games_of_player_train(i):
    return [games_list[index] for index in list(np.asarray(X_train.iloc[i]).nonzero())[0]]

def games_of_player_test(i):
    return [games_list[index] for index in list(np.asarray(X_test.iloc[i]).nonzero())[0]]

neigh = NearestNeighbors(n_neighbors=5, metric='euclidean')
neigh.fit(X_train.values)

NearestNeighbors(metric='euclidean')

We can now create our recommendation function :  

Taking :  
- a player X, ie a vector of hours played for each game  
- a knn model fitted on values as shown before  
- the samples used for training

It returns :  
- the list of games liked by the k neighbors that X never played

In [32]:
def recommendations(X, model, samples):
    #Get the list of games played by X
    already_owned = [games_list[index] for index in np.asarray(X).nonzero()[0]]
    
    #Returns the union of games liked by neighbors 
    kneighbors = model.kneighbors([X])[1][0] #We get the indices of the neighbors
    
    recommandations = []
    for neighbor in kneighbors:
        recommandations = recommandations+[games_list[index] for index in np.asarray(samples.iloc[neighbor]).nonzero()[0]]
    
    #remove duplicates
    recommandations = list(dict.fromkeys(recommandations))
    
    #remove games already owned by X
    recommandations = [game for game in recommandations if game not in already_owned]
    
    return recommandations

In [33]:
games_of_player_test(250)

['Counter-Strike Global Offensive', 'Dota 2', 'PAYDAY The Heist']

In [34]:
recommendations(X_test.iloc[250], neigh, y_train)

['Far Cry 3',
 "Sid Meier's Civilization V",
 'Star Wars - Battlefront II',
 'Sniper Elite V2',
 'Unturned',
 'War Thunder']

In [35]:
#Let's create a test rpg player to test the recommendations
test_player = {'The Witcher 3 Wild Hunt':132,'The Elder Scrolls V Skyrim':224, 'Far Cry':90}
encoded_player = pd.Series(test_player,index=games_list).fillna(0)
encoded_player

007 Legends                                                   0.0
0RBITALIS                                                     0.0
1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby)    0.0
10 Second Ninja                                               0.0
10,000,000                                                    0.0
                                                             ... 
rymdkapsel                                                    0.0
sZone-Online                                                  0.0
the static speaks my name                                     0.0
theHunter                                                     0.0
theHunter Primal                                              0.0
Length: 3564, dtype: float64

In [36]:
recommendations(encoded_player, neigh, y_train)

['Anno 2070',
 "Assassin's Creed II",
 'BioShock',
 'Borderlands 2',
 'Call of Duty 4 Modern Warfare',
 'Call of Duty Black Ops',
 'Call of Duty Modern Warfare 2',
 'Call of Duty Modern Warfare 3',
 'Call of Duty World at War',
 'DC Universe Online',
 'Dungeons & Dragons Daggerdale',
 'Endless Space',
 'Fable III',
 'Fallout 4',
 'Grey Goo',
 'Homefront',
 'Medal of Honor(TM) Single Player',
 'Portal',
 'Shadowrun Returns',
 'Star Trek Online',
 'Sunless Sea',
 'Tomb Raider',
 'Toy Soldiers',
 'BRINK',
 'Call of Duty Ghosts',
 'Call of Juarez Gunslinger',
 'Fallout 3 - Game of the Year Edition',
 'Hitman Absolution',
 'Middle-earth Shadow of Mordor',
 'Resident Evil Revelations / Biohazard Revelations',
 'Risen 2 - Dark Waters',
 'Stronghold 3',
 'Thief',
 'Wolfenstein The New Order German Edition',
 'ArcaniA',
 'Half-Life 2 Episode One',
 'The Darkness II',
 'A Game of Thrones - Genesis',
 'Age of Mythology Extended Edition',
 'Banished',
 'DUNGEONS - Steam Special Edition',
 'Dungeon

We get results that seem coherent (for this example we have same genre like rpgs, also series of games already played), but there are too much of them.  

To prevent this, we will firstly give a score to each result to determine the priority of recommendations, and then define how much maximum results we want.  

**Score**  
The score of a predicted game will be equal to the sum of the inverse distances to each neighbor, weighted by how much the neighbors liked the game (the difference between the hours played and the mean).  
With this score, the more a game has been played by people who like the same games and the more it has been liked by them, the more it will have chances to be predicted.  

To do so, we set to 0 the negative values of hours, meaning that we only keep games likes and the value will be proportional to the difference to the average.

In [37]:
def dict_to_likeness(dicti):
    d = dicti.copy()
    for game in d.keys():
        if d[game] < 0:
            d[game]= 0
    return d

#For each player that played at least 3 games, we compute the list of game he like
likeness_games = played_dict_3.map(dict_to_likeness)
likeness_games

id
5250         {'Cities Skylines': 3.158903786773205, 'Deus E...
76767        {'Counter-Strike': 0.20796408562479032, 'Call ...
86540        {'The Elder Scrolls V Skyrim': 0.0478209339091...
229911       {'Counter-Strike Condition Zero': 1.1203642472...
298950       {'Team Fortress 2': 3.045243405037884, 'Counte...
                                   ...                        
306547522    {'Warface': 0, 'Dota 2': 0, 'Strife': 0, 'Team...
306971738    {'The Elder Scrolls IV Oblivion ': 0, 'Terrari...
308695132    {'La Tale': 0, 'Champions Online': 0, 'Brawlha...
308760273    {'Toribash': 0, 'Unturned': 0, 'Team Fortress ...
309404240    {'Unturned': 0, 'Team Fortress 2': 0, 'Mitos.i...
Length: 3466, dtype: object

The new recommendation function will then return a vector of probabilities that the player may like each game.

In [38]:
#And encode them
likeness_games_encoded = likeness_games.apply(pd.Series)
#Replace NaN values by 0 : a game not in the dict has never been played
likeness_games_encoded = likeness_games_encoded.fillna(0)
likeness_games_encoded = likeness_games_encoded.reindex(sorted(likeness_games_encoded.columns),axis=1)

In [39]:
#We define a custom train test split function to shuffle the likeness with the two other dataframes
def train_test_split_likeness(hours, liked_games, likeness, test_size=0.2):
    
    #shuffle
    indexes = np.asarray(hours.index).copy()
    np.random.seed(42) #set seed for reproducability
    np.random.shuffle(indexes)
    hours_shuffled = hours.loc[indexes]
    liked_games_shuffled = liked_games.loc[indexes]
    likeness_shuffled = likeness.loc[indexes]
    
    #split train and test
    separator = len(hours) - int(len(hours)* test_size)
    X_train = hours_shuffled.iloc[:separator]
    y_train = liked_games_shuffled.iloc[:separator]
    X_test = hours_shuffled.iloc[separator:]
    y_test = liked_games_shuffled.iloc[separator:]
    
    return X_train,X_test,y_train,y_test,likeness_shuffled

In [40]:
X_train, X_test, y_train, y_test, likeness = train_test_split_likeness(hours_encoded_3, liked_games_encoded, likeness_games_encoded, test_size=0.2)


#To retrieve the list of played games, we get the list of games with a
#non zero time played and check the name corresponding to the index
def games_of_player_train(i):
    return [games_list[index] for index in list(np.asarray(X_train.iloc[i]).nonzero())[0]]

def games_of_player_test(i):
    return [games_list[index] for index in list(np.asarray(X_test.iloc[i]).nonzero())[0]]

neigh = NearestNeighbors(n_neighbors=5, metric='euclidean')
neigh.fit(X_train.values)

NearestNeighbors(metric='euclidean')

In [41]:
def score(X, model, likeness):
    #Create a vector with all games and a null score
    score = pd.Series({games_list[0]:0.0},index=games_list).fillna(0)
    
    #Get the list of games played by X
    already_owned = [games_list[index] for index in np.asarray(X).nonzero()[0]]
    
    #Get the neighbors of X
    kneighbors = model.kneighbors([X])
    kneighbors_distances = kneighbors[0][0]
    kneighbors_indices = kneighbors[1][0]
    
    for i in range(len(kneighbors_indices)):
        neighbor = kneighbors_indices[i]
        #get the list of liked games
        liked = [games_list[index] for index in np.asarray(likeness.iloc[neighbor]).nonzero()[0]]
        #Add to each game score (1/d)*l with d the distance between X and the neighbor
        #and l the amount of time played above the average
        for liked_game in liked:
            if liked_game not in already_owned:
                score[liked_game] = score[liked_game] + 1/kneighbors_distances[i] + likeness.iloc[neighbor][liked_game]
            
    
    return score.sort_values(ascending=False)

In [42]:
dict(score(encoded_player, neigh, likeness))

{'Medal of Honor(TM) Single Player': 5.167203361811293,
 "Tom Clancy's Splinter Cell Blacklist": 3.8767318856965547,
 'Rome Total War': 3.8553351998618384,
 'Middle-earth Shadow of Mordor': 3.3553337597994486,
 'A Game of Thrones - Genesis': 3.199345188988867,
 'Star Wars Empire at War Gold': 3.0703757349142924,
 'Call of Duty 4 Modern Warfare': 2.8868990897913793,
 'Stronghold 3': 2.8334860199370677,
 'Fallout 3 - Game of the Year Edition': 2.6444250227878716,
 'Mafia II': 2.6437238618602263,
 'S.T.A.L.K.E.R. Call of Pripyat': 2.617017972398454,
 'Sunless Sea': 2.5793778491182215,
 'Thief': 2.39538664157769,
 'Hitman Absolution': 2.3017264957209607,
 'Deus Ex Human Revolution': 2.2377269594122953,
 'Toy Soldiers': 2.2141294323955196,
 "Tom Clancy's Splinter Cell": 2.2118791275287832,
 'Dead Space 2': 2.2034767543082423,
 'Endless Space': 2.0099696270999594,
 'Patrician IV Steam Special Edition': 1.974416453577219,
 'Dungeons & Dragons Daggerdale': 1.8213465374510696,
 'Tomb Raider': 1

With this score, the results are much more satisfying !

Now that our method is set, let's create a model class to clean everything

In [43]:
class SteamPredictionModel():
    
    def __init__(self, k_neighbors = 5):
        self.neigh = NearestNeighbors(n_neighbors=k_neighbors, metric='euclidean')
        self.games_list = []
        self.avg_dict = {}
        self.likeness = None
        
    def dict_to_likeness(self,dicti):
        d = dicti.copy()
        for game in d.keys():
            if d[game] < 0:
                d[game]= 0
        return d
    
    #We fit the model on a dataset containing ids and dictionnaries of games associated with time played
    def fit(self, data):
        hours_encoded = data.apply(pd.Series)
        #Replace NaN values by 0 : a game not in the dict has never been played
        hours_encoded = hours_encoded.fillna(0)
        hours_encoded = hours_encoded.reindex(sorted(hours_encoded.columns),axis=1)
        
        self.neigh.fit(hours_encoded.values)
        
        #Get the list of games
        games_list = list(hours_encoded.columns)
        games_list.sort()
        self.games_list = games_list
        
        #For each player, we compute the list of game he likes with the time he has played aboved average time played
        likeness_games = data.map(self.dict_to_likeness)
        #And encode them
        likeness_games_encoded = likeness_games.apply(pd.Series)
        #Replace NaN values by 0 : a game not in the dict has never been played
        likeness_games_encoded = likeness_games_encoded.fillna(0)
        likeness_games_encoded = likeness_games_encoded.reindex(sorted(likeness_games_encoded.columns),axis=1)
        self.likeness = likeness_games_encoded
    
    #We predict a certain number of games (maximum) using a dictionnary of games associated with time played
    def predict(self, X_init, recommendations_number_max):
        #One-hot-encode X
        X = score = pd.Series(X_init,index=self.games_list).fillna(0)
        
        #Create a vector with all games and a null score
        score = pd.Series({self.games_list[0]:0.0},index=self.games_list).fillna(0)

        #Get the list of games played by X
        already_owned = [self.games_list[index] for index in np.asarray(X).nonzero()[0]]

        #Get the neighbors of X
        kneighbors = self.neigh.kneighbors([X])
        kneighbors_distances = kneighbors[0][0]
        kneighbors_indices = kneighbors[1][0]

        for i in range(len(kneighbors_indices)):
            neighbor = kneighbors_indices[i]
            #get the list of liked games
            liked = [self.games_list[index] for index in np.asarray(self.likeness.iloc[neighbor]).nonzero()[0]]
            #Add to each game score (1/d)*l with d the distance between X and the neighbor
            #and l the amount of time played above the average
            for liked_game in liked:
                if liked_game not in already_owned:
                    score[liked_game] = score[liked_game] + 1/kneighbors_distances[i] + self.likeness.iloc[neighbor][liked_game]


        score = score.sort_values(ascending=False)
        return score.iloc[:recommendations_number_max]

In [44]:
SPM = SteamPredictionModel(20)
SPM.fit(played_dict_3)

In [45]:
SPM.predict(test_player,20)

Just Cause 2                      11.437667
Borderlands 2                     11.068734
Far Cry 3                         10.472361
Saints Row The Third              10.142682
Surgeon Simulator                  7.690374
Chivalry Medieval Warfare          7.520081
Middle-earth Shadow of Mordor      7.351925
Half-Life 2 Lost Coast             6.965404
Deus Ex Human Revolution           6.564230
Need for Speed Hot Pursuit         6.365282
Grand Theft Auto San Andreas       6.320411
Far Cry 3 Blood Dragon             6.305465
Saints Row IV                      6.213651
The Elder Scrolls IV Oblivion      5.844674
Arma 2                             5.804962
Trine                              5.678657
Worms Reloaded                     5.642036
Wolfenstein The New Order          5.632307
Assassin's Creed II                5.288043
Saints Row 2                       5.279066
dtype: float64

## Knn evaluation

The evaluation metrics we have seen in class are fitted to classifiers evaluation. 
To use them, we will evaluate our algorithm using the testing set like this:  
- 1. remove 5% of the games from the player's played games (the most played)
- 2. use the algorithm to predict the number of games lacking
- 3. compare the recommendation to the games that have been removed in step 1  

The result of the evaluation is: 
- true positive if the game predicted was in the games removed
- false positive if the game predicted was not in the games removed

The number of false negatives (not to predict a game that has been removed) is then exactly equals to the number of false positives, and the number of true negatives is irrelevant here.

That's why we will use the **precision** metrics to evaluate our model.

First, let's split our data into random training and testing sets

In [46]:
def train_test_split(data, test_size=0.2):
    
    #shuffle
    shuffled = data.sample(frac=1, random_state=42) #set seed for reproducability
    
    #split train and test
    separator = len(shuffled) - int(len(shuffled)* test_size)
    X_train = shuffled.iloc[:separator]
    X_test = shuffled.iloc[separator:]
    
    return X_train,X_test

X_train, X_test = train_test_split(played_dict_3)

In [47]:
X_test.iloc[0]

{'Call of Duty Modern Warfare 3 - Multiplayer': -0.3237429079056385,
 'Darksiders': 0.3716968613154826,
 'RAGE': 0.10430638645293915,
 'Call of Duty Modern Warfare 3': -0.34071597864459907,
 'Dungeon Siege III': -0.4682173912038673,
 'Call of Juarez The Cartel': -0.6505653608433958,
 'Nosgoth': -0.29489055229921474,
 'Kane & Lynch 2 Dog Days': -0.609001797637538}

In [48]:
#Create and fit the model
SPM = SteamPredictionModel(20)
SPM.fit(X_train)

In [49]:
from tqdm import tqdm

#Evaluate the model
true_positives = 0
false_positives = 0

for player in tqdm(X_test):
    
    player_games = len(player)
    
    games_to_predict = int(len(player)*5/100)
    if games_to_predict == 0:
        games_to_predict = 1 #we remove at least one game
    
    games_for_prediction = {}
    games_removed = []
    n = 0
    for (name,time) in list(player.items()):
        if n < games_to_predict and time >0: #the game is liked
            games_removed.append(name)
            n += 1
        else:
            games_for_prediction[name]=time
    
    prediction = []
    if len(games_removed) > 0:
        prediction = list(SPM.predict(games_for_prediction,games_to_predict).index)
    
    for predicted_game in prediction:
        if predicted_game in games_removed:
            true_positives += 1
        else:
            false_positives += 1

precision = true_positives/(true_positives+false_positives)

100%|█████████████████████████████████████████| 693/693 [00:20<00:00, 33.61it/s]


In [50]:
true_positives, false_positives, precision

(60, 700, 0.07894736842105263)

The result is obviously extremely underwhelming. Indeed, given a random game, it is highly probable that a random person would not like it, and thus gives this result. Moreover, the model tends to work better for people that have played a lot of games, and as we're using data from people that may have played only 3 games, predicting 33% of all the games played by someone is really hard.

Let's try different values of the number of neighbors.

In [ ]:
precisions = []
for n in tqdm(range(1,31)):
    #Create and fit the model
    SPM = SteamPredictionModel(n)
    SPM.fit(X_train)
    
    true_positives = 0
    false_positives = 0

    for player in X_test:

        player_games = len(player)

        games_to_predict = int(len(player)*5/100)
        if games_to_predict == 0:
            games_to_predict = 1 #we remove at least one game

        games_for_prediction = {}
        games_removed = []
        n = 0
        for (name,time) in list(player.items()):
            if n < games_to_predict and time >0: #the game is liked
                games_removed.append(name)
                n += 1
            else:
                games_for_prediction[name]=time

        prediction = []
        if len(games_removed) > 0:
            prediction = list(SPM.predict(games_for_prediction,games_to_predict).index)

        for predicted_game in prediction:
            if predicted_game in games_removed:
                true_positives += 1
            else:
                false_positives += 1
    precisions.append(true_positives/(true_positives+false_positives))

 87%|█████████████████████████████████████▎     | 26/30 [08:13<01:23, 20.86s/it]

In [ ]:
plt.plot(range(1,31),precisions)
plt.title('Precision in function of k')

In [ ]:
played_dict_5 = played_dict.loc[played_dict.map(len)>=5]
len(played_dict_5)

In [ ]:
X_train, X_test = train_test_split(played_dict_5)

In [ ]:
#Create and fit the model
SPM = SteamPredictionModel(20)
SPM.fit(X_train)

In [ ]:
from tqdm import tqdm

#Evaluate the model
true_positives = 0
false_positives = 0

for player in tqdm(X_test):
    
    player_games = len(player)
    
    games_to_predict = 1 # we predict only 1 game
    
    games_for_prediction = {}
    games_removed = []
    n = 0
    for (name,time) in list(player.items()):
        if n < games_to_predict and time >0: #the game is liked
            games_removed.append(name)
            n += 1
        else:
            games_for_prediction[name]=time
    
    prediction = list(SPM.predict(games_for_prediction,10).index) #we predict 3 games
    
    if len(games_removed) > 0:
        if games_removed[0] in prediction:
            true_positives += 1
        else:
            false_positives += 1

precision = true_positives/(true_positives+false_positives)

In [ ]:
true_positives, false_positives, precision